# Voting

In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Importing the dataset
dataset = pd.read_csv('CVD dataset2.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 16].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
17


In [2]:
dataNoCvd = data[~(data[:,16] > 0.0)]
dataCvd = data[~(data[:,16] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [3]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [22]:
def create_train_test_sets(cv,dataNoCvd,dataCvd):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)

    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)

        testSubset_NoCvd ,testSubset_Cvd ,testSubset_total = find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
     
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, 16, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, 16), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
    
        X_train.append(np.delete(trainSubset_total, 16, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, 16), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
#         print(X_test)
#         print(y_test)
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, train_total_NoCvd, train_total_Cvd

In [5]:
train_total, test_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd)
print(train_total_NoCvd)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9
[array([[  1.  ,  50.  ,  14.  ,  25.16,   2.  ,   1.  ,   1.  ,   0.  ,
          1.  ,   1.  ,  45.  ,   9.6 , 215.  , 251.  , 228.  ,  47.  ,
          0.  ],
       [  1.  ,  70.  ,   1.  ,  24.27,   1.  ,   3.  ,   2.  ,   0.  ,
          1.  ,   1.  ,  75.  ,   4.5 , 110.  , 222.  , 153.  ,  52.  ,
          0.  ],
       [  2.  ,  58.  ,   7.  ,  36.93,   1.  ,   3.  ,   2.  ,   0.  ,
          1.  ,   1.  ,  70.  ,   6.9 , 160.  , 205.  , 126.  ,  39.  ,
          0.  ],
       [  2.  ,  70.  ,  13.  ,  33.32,   1.  ,   1.  ,   2.  ,   0.  ,
          1.  ,   1.  , 105.  ,   7.1 , 125.  , 316.  , 215.  ,  61.  ,
          0.  ],
       [  1.  ,  44.  ,   5.  ,  25.06,   1.  ,   1.  ,   1.  ,   0.  ,
          1.  ,   1.  ,  45.  ,   6.6 , 160.  , 187.  , 154.  ,  46.  ,
          0.  ],
       [  1.  ,  38.  ,   4.  ,  38.86,   2.  ,   3.  ,   1.  ,   0.  ,
          1.  ,   1.  ,  90.  ,   7.

In [6]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)

504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07142857142857142
ratio in test set: 0.08928571428571429


In [69]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

def my_fit(ratio,dataNoCvd,dataCvd,params):   
    Subarray=[]
    models = []
    # yarray


    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
    
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0
    
    classifier = RandomForestClassifier(n_estimators=params[0],max_depth=params[1],max_features=params[2],
                     min_samples_leaf=params[3],min_samples_split=params[4],criterion=params[5])

                
    for i in range(numOfSubsamples):
        #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], 16, axis=1)
        y=np.delete(Subarray[i], slice(0, 16), axis=1)
        y=np.reshape(y, len(y))
        
        
    #     print(i)
    
        classifier.fit( X, y)
        models.append(classifier)
    return models     

In [ ]:
results = []
        results1 = []
#         n_estimator_options = []
#         for k in range(20):
#             n_estimator_options.append(k+1)
        n_estimator_options = [5,7,8,9,10,11,12,13,14, 15,18, 20,22,25,28,30, 40, 50, 70, 100]

        for trees in n_estimator_options:
            classifier = RandomForestClassifier(trees, oob_score=True)
            #model.fit(X, y)
            classifier.fit(X, y)
            print(trees, 'trees')
            AUC = classifier.oob_score_
            print('ACC: ', AUC)
            results.append(AUC)
            
            
            pred_train = np.argmax(classifier.oob_decision_function_,axis=1)
            print('AUC: ',(metrics.roc_auc_score(y, pred_train)))
            results1.append(metrics.roc_auc_score(y, pred_train))
            print (" ")
            
#         plt.plot(n_estimator_options, results, label='OOB Score')
#         plt.xlabel('# of Trees')
#         plt.ylabel('OOB Score')
#         plt.title('OOB Score VS Trees')
#         plt.legend(loc="lower right")
#         plt.show()
        
        plt.plot(n_estimator_options, results1, label='OOB Score')
        plt.xlabel('# of Trees')
        plt.ylabel('OOB Score')
        plt.title('OOB Score VS Trees')
        plt.legend(loc="lower right")
        plt.show()

In [54]:
models = my_fit(2,dataNoCvd,dataCvd)   

TypeError: my_fit() missing 1 required positional argument: 'params'

In [8]:

def voting(models,X):
    y_pred = []
    y_pred_models = []

    models_size = len(models)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict(X))
        
    for j in range(X_size):
        case = 0
        for i in range(models_size):
            if y_pred_models[i][j] == 1.0:
                case+=1
        if case >  models_size/2 :
            y_pred.append(1.) 
        else:
            y_pred.append(0.) 
            
    return y_pred        
     

In [24]:
y_pred = voting(models,test_total[9][0])

In [25]:
print(y_pred)
# print(y_pred_models[0])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [25]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, test_total, train_total_NoCvd, train_total_Cvd,params):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 2, train_total_NoCvd[i], train_total_Cvd[i],params)


        y_pred = voting(models,X_test)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [58]:
params = {
                    "learning_rate"    :[ 0.1],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[200],
                     "subsample"       :[1.0],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[0.5], #0.5-1
                     "reg_lambda"      :[1.5], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[3]
} 

In [59]:
comb = compute_comb(params)
print(comb[0])
accuracy, specificity, sensitivity, auc = my_cross_val( 10, test_total, train_total_NoCvd, train_total_Cvd,comb[0])

[0.1, 2, 200, 1.0, 0, 0.5, 1.5, 6]


In [60]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

0.725
0.7496606334841629
0.415
0.5823303167420815


In [70]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("n_estimators")
    list2 = params.get("max_depth")
    list3 = params.get("max_features") 
    list4 = params.get("min_samples_leaf")
    list5 = params.get("min_samples_split")  
    list6 = params.get("criterion")
    


#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n] for i in list1  
                                    for j in list2 
                                    for k in list3 
                                    for l in list4 
                                    for m in list5 
                                    for n in list6]
 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [ ]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("n_estimators")
    list2 = params.get("max_depth")
    list3 = params.get("max_features") 
    list4 = params.get("min_samples_leaf")
    list5 = params.get("min_samples_split")  
    list6 = params.get("criterion")
    
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all possible permutations 
    res = [[i, j, k, l, m, n, o, p] for i in list1  
                                    for j in list2 
                                    for k in list3 
                                    for l in list4 
                                    for m in list5 
                                    for n in list6
                                    for o in list7
                                    for p in list8] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [11]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [15]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
    print(len(combs))
    for j in range(len(combs)):
        print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, test_total, train_total_NoCvd, train_total_Cvd,combs[j])
        if mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
        else:
            print(combs[j])
            print("accuracy    ",mean(accuracy))
            print("specificity ",mean(specificity))
            print("sensitivity ",mean(sensitivity))
            print("auc         ",mean(auc))
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc)])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params

# Ratio = 2

In [67]:
     
params = {     'n_estimators': [5,8,10,12,14,15,18,20,22,25,28,30,40, 50, 70, 100],
               'max_depth':    [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10], 
               'criterion': ['gini','entropy']
        }


In [74]:
params = {     
               'n_estimators': [10,20,50,70,100],
               'max_depth':    [20, 30, 40, None],
               'max_features': ['auto', 'sqrt'],
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': [2, 5, 10], 
               'criterion': ['gini','entropy']
        }


Στην συνέχεια τρέχουμε την gridsearch και εμφανίζουμε όλα τα αποτελέσματα. Μεταξύ γραμμών και με bold τυπώνονται οι τιμές που ικανοπούν τις συνθήκες:
    mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5:

In [75]:
max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)

720
0
[10, 20, 'auto', 1, 2, 'gini']
accuracy     0.7321428571428572
specificity  0.7670814479638008
sensitivity  0.295
auc          0.5310407239819005
1
[10, 20, 'auto', 1, 2, 'entropy']
accuracy     0.775
specificity  0.8095022624434389
sensitivity  0.34
auc          0.5747511312217195
2
[10, 20, 'auto', 1, 5, 'gini']
accuracy     0.7410714285714286
specificity  0.7806184012066365
sensitivity  0.245
auc          0.5128092006033182
3
[10, 20, 'auto', 1, 5, 'entropy']
accuracy     0.7767857142857143
specificity  0.8151960784313725
sensitivity  0.285
auc          0.5500980392156862
4
[10, 20, 'auto', 1, 10, 'gini']
accuracy     0.7285714285714285
specificity  0.755656108597285
sensitivity  0.395
auc          0.5753280542986425
5
[10, 20, 'auto', 1, 10, 'entropy']
accuracy     0.7214285714285714
specificity  0.7556184012066365
sensitivity  0.29
auc          0.5228092006033183
6
[10, 20, 'auto', 2, 2, 'gini']
accuracy     0.7446428571428572
specificity  0.778657616892911
sensitivity  0.32

[10, 30, 'sqrt', 1, 2, 'entropy']
accuracy     0.7785714285714286
specificity  0.8248868778280543
sensitivity  0.195
auc          0.5099434389140272
56
[10, 30, 'sqrt', 1, 5, 'gini']
accuracy     0.7339285714285715
specificity  0.757579185520362
sensitivity  0.445
auc          0.601289592760181
57
[10, 30, 'sqrt', 1, 5, 'entropy']
accuracy     0.775
specificity  0.8095022624434389
sensitivity  0.345
auc          0.5772511312217194
58
[10, 30, 'sqrt', 1, 10, 'gini']
accuracy     0.7357142857142858
specificity  0.7690422322775264
sensitivity  0.315
auc          0.5420211161387631
59
[10, 30, 'sqrt', 1, 10, 'entropy']
accuracy     0.7803571428571429
specificity  0.8133861236802413
sensitivity  0.37
auc          0.5916930618401207
60
[10, 30, 'sqrt', 2, 2, 'gini']
accuracy     0.7428571428571429
specificity  0.7788084464555053
sensitivity  0.3
auc          0.5394042232277526
61
[10, 30, 'sqrt', 2, 2, 'entropy']
accuracy     0.7321428571428572
specificity  0.7671191553544494
sensitivity  0.

[10, None, 'auto', 1, 5, 'entropy']
accuracy     0.7303571428571428
specificity  0.7687782805429865
sensitivity  0.24
auc          0.5043891402714932
112
[10, None, 'auto', 1, 10, 'gini']
accuracy     0.7535714285714286
specificity  0.7863499245852187
sensitivity  0.34
auc          0.5631749622926093
113
[10, None, 'auto', 1, 10, 'entropy']
accuracy     0.7517857142857143
specificity  0.7826168929110106
sensitivity  0.37
auc          0.5763084464555053
114
[10, None, 'auto', 2, 2, 'gini']
accuracy     0.7660714285714285
specificity  0.7941176470588235
sensitivity  0.42
auc          0.6070588235294118
115
[10, None, 'auto', 2, 2, 'entropy']
accuracy     0.7696428571428572
specificity  0.7959276018099547
sensitivity  0.435
auc          0.6154638009049774
116
[10, None, 'auto', 2, 5, 'gini']
accuracy     0.7375
specificity  0.7786953242835596
sensitivity  0.22
auc          0.4993476621417798
117
[10, None, 'auto', 2, 5, 'entropy']
accuracy     0.7625
specificity  0.7922699849170437
sensit

[20, 20, 'sqrt', 1, 10, 'gini']
accuracy     0.7303571428571428
specificity  0.7634238310708898
sensitivity  0.32
auc          0.541711915535445
167
[20, 20, 'sqrt', 1, 10, 'entropy']
accuracy     0.7589285714285714
specificity  0.7922699849170437
sensitivity  0.345
auc          0.5686349924585219
168
[20, 20, 'sqrt', 2, 2, 'gini']
accuracy     0.75
specificity  0.7864630467571644
sensitivity  0.295
auc          0.5407315233785822
169
[20, 20, 'sqrt', 2, 2, 'entropy']
accuracy     0.7553571428571428
specificity  0.7883861236802413
sensitivity  0.345
auc          0.5666930618401207
170
[20, 20, 'sqrt', 2, 5, 'gini']
accuracy     0.775
specificity  0.8133107088989442
sensitivity  0.3
auc          0.5566553544494721
171
[20, 20, 'sqrt', 2, 5, 'entropy']
accuracy     0.7464285714285714
specificity  0.7826168929110106
sensitivity  0.295
auc          0.5388084464555053
172
[20, 20, 'sqrt', 2, 10, 'gini']
accuracy     0.7321428571428571
specificity  0.7673076923076924
sensitivity  0.295
auc  

[20, 40, 'auto', 1, 10, 'entropy']
accuracy     0.7625
specificity  0.798001508295626
sensitivity  0.32
auc          0.559000754147813
222
[20, 40, 'auto', 2, 2, 'gini']
accuracy     0.7714285714285715
specificity  0.8095776772247361
sensitivity  0.295
auc          0.552288838612368
223
[20, 40, 'auto', 2, 2, 'entropy']
accuracy     0.7428571428571429
specificity  0.7749245852187029
sensitivity  0.34
auc          0.5574622926093514
224
[20, 40, 'auto', 2, 5, 'gini']
accuracy     0.7535714285714286
specificity  0.7806184012066365
sensitivity  0.415
auc          0.5978092006033182
225
[20, 40, 'auto', 2, 5, 'entropy']
accuracy     0.7732142857142857
specificity  0.8076546003016591
sensitivity  0.34
auc          0.5738273001508295
226
[20, 40, 'auto', 2, 10, 'gini']
accuracy     0.7428571428571429
specificity  0.7730392156862745
sensitivity  0.37
auc          0.5715196078431373
227
[20, 40, 'auto', 2, 10, 'entropy']
accuracy     0.7821428571428571
specificity  0.8172699849170437
sensitivi

...

[70, 30, 'sqrt', 4, 2, 'entropy']
accuracy     0.7928571428571428
specificity  0.8346153846153846
sensitivity  0.27
auc          0.5523076923076923
500
[70, 30, 'sqrt', 4, 5, 'gini']
accuracy     0.7839285714285714
specificity  0.8249622926093514
sensitivity  0.27
auc          0.5474811463046757
501
[70, 30, 'sqrt', 4, 5, 'entropy']
accuracy     0.7928571428571428
specificity  0.8326546003016592
sensitivity  0.295
auc          0.5638273001508295
502
[70, 30, 'sqrt', 4, 10, 'gini']
accuracy     0.7964285714285715
specificity  0.8365384615384616
sensitivity  0.295
auc          0.5657692307692308
503
[70, 30, 'sqrt', 4, 10, 'entropy']
accuracy     0.8017857142857143
specificity  0.8423076923076923
sensitivity  0.295
auc          0.5686538461538462
504
[70, 40, 'auto', 1, 2, 'gini']
accuracy     0.7696428571428571
specificity  0.8018853695324284
sensitivity  0.365
auc          0.5834426847662142
505
[70, 40, 'auto', 1, 2, 'entropy']
accuracy     0.775
specificity  0.8173076923076923
sensit

[70, None, 'auto', 4, 5, 'gini']
accuracy     0.7946428571428572
specificity  0.8384615384615385
sensitivity  0.245
auc          0.5417307692307692
555
[70, None, 'auto', 4, 5, 'entropy']
accuracy     0.8107142857142857
specificity  0.8557692307692308
sensitivity  0.245
auc          0.5503846153846154
556
[70, None, 'auto', 4, 10, 'gini']
accuracy     0.8089285714285714
specificity  0.8538461538461538
sensitivity  0.245
auc          0.5494230769230769
557
[70, None, 'auto', 4, 10, 'entropy']
accuracy     0.7910714285714285
specificity  0.832654600301659
sensitivity  0.27
auc          0.5513273001508295
558
[70, None, 'sqrt', 1, 2, 'gini']
accuracy     0.7660714285714286
specificity  0.8018476621417798
sensitivity  0.32
auc          0.5609238310708899
559
[70, None, 'sqrt', 1, 2, 'entropy']
accuracy     0.775
specificity  0.8134238310708899
sensitivity  0.295
auc          0.5542119155354449
560
[70, None, 'sqrt', 1, 5, 'gini']
accuracy     0.7714285714285715
specificity  0.8076923076923

[100, 20, 'sqrt', 4, 5, 'entropy']
accuracy     0.8160714285714286
specificity  0.8538084464555052
sensitivity  0.345
auc          0.5994042232277527
610
[100, 20, 'sqrt', 4, 10, 'gini']
accuracy     0.8053571428571429
specificity  0.8480392156862745
sensitivity  0.27
auc          0.5590196078431372
611
[100, 20, 'sqrt', 4, 10, 'entropy']
accuracy     0.7982142857142858
specificity  0.8403846153846154
sensitivity  0.27
auc          0.5551923076923078
612
[100, 30, 'auto', 1, 2, 'gini']
accuracy     0.7767857142857143
specificity  0.8114630467571644
sensitivity  0.345
auc          0.5782315233785822
613
[100, 30, 'auto', 1, 2, 'entropy']
accuracy     0.775
specificity  0.8153092006033182
sensitivity  0.27
auc          0.5426546003016591
614
[100, 30, 'auto', 1, 5, 'gini']
accuracy     0.7625
specificity  0.7980769230769231
sensitivity  0.32
auc          0.5590384615384616
615
[100, 30, 'auto', 1, 5, 'entropy']
accuracy     0.7732142857142857
specificity  0.8115384615384615
sensitivity  

[100, 40, 'auto', 4, 10, 'gini']
accuracy     0.7803571428571429
specificity  0.8192307692307692
sensitivity  0.295
auc          0.5571153846153846
665
[100, 40, 'auto', 4, 10, 'entropy']
accuracy     0.7928571428571428
specificity  0.8326923076923077
sensitivity  0.295
auc          0.5638461538461539
666
[100, 40, 'sqrt', 1, 2, 'gini']
accuracy     0.775
specificity  0.8076546003016591
sensitivity  0.37
auc          0.5888273001508295
667
[100, 40, 'sqrt', 1, 2, 'entropy']
accuracy     0.7839285714285714
specificity  0.8211161387631976
sensitivity  0.32
auc          0.5705580693815988
668
[100, 40, 'sqrt', 1, 5, 'gini']
accuracy     0.7571428571428571
specificity  0.7883861236802413
sensitivity  0.37
auc          0.5791930618401206
669
[100, 40, 'sqrt', 1, 5, 'entropy']
accuracy     0.7678571428571429
specificity  0.8018099547511313
sensitivity  0.34
auc          0.5709049773755657
670
[100, 40, 'sqrt', 1, 10, 'gini']
accuracy     0.7464285714285714
specificity  0.7788461538461539
sen

[100, None, 'sqrt', 4, 10, 'entropy']
accuracy     0.8
specificity  0.8423076923076923
sensitivity  0.27
auc          0.5561538461538461


In [77]:
print(max_params)
print(max_acc)
print(max_spes)
print(max_sens)
print(max_auc)

[10, 40, 'auto', 2, 5, 'entropy']
0.7678571428571428
0.7921191553544494
0.47
0.6310595776772248


In [ ]:
[10, 20, 'auto', 2, 5, 'gini']
accuracy     0.725
specificity  0.7421568627450981
sensitivity  0.515
auc          0.628578431372549

In [78]:
#Τυπώνονται περιπτώσεις που auc>0.6
s = 0
for i in range(len(best_params)):
    if best_params[i][4] > 0.61:
        print("parameters :",best_params[i][0])
        print("accuracy :",best_params[i][1])
        print("spes     :",best_params[i][2])
        print("sens     :",best_params[i][3])
        print("auc      :",best_params[i][4])
        print("------------------------------------------------")
        s+=1
print(s)

parameters : [10, 20, 'auto', 2, 5, 'gini']
accuracy : 0.725
spes     : 0.7421568627450981
sens     : 0.515
auc      : 0.628578431372549
------------------------------------------------
parameters : [10, 40, 'auto', 2, 5, 'entropy']
accuracy : 0.7678571428571428
spes     : 0.7921191553544494
sens     : 0.47
auc      : 0.6310595776772248
------------------------------------------------
parameters : [10, 40, 'auto', 4, 5, 'entropy']
accuracy : 0.7589285714285714
spes     : 0.7826546003016591
sens     : 0.465
auc      : 0.6238273001508295
------------------------------------------------
parameters : [10, None, 'auto', 2, 2, 'entropy']
accuracy : 0.7696428571428572
spes     : 0.7959276018099547
sens     : 0.435
auc      : 0.6154638009049774
------------------------------------------------
4
